In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

from pydataset import data

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import RFE, SelectKBest, f_regression
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from useful_functions import get_object_cols, get_numeric_cols
from wrangle import split_data, scale_data, X_y_splitb

ImportError: cannot import name 'X_y_splitb' from 'wrangle' (/Users/derekbixby/codeup-data-science/regression_exercises/wrangle.py)

In [3]:
def X_y_split(df, target):
    '''
    This function takes in a dataframe and a target variable
    Then it returns the X_train, y_train, X_validate, y_validate, X_test, y_test
    and a print statement with the shape of the new dataframes
    '''  
    train, validate, test = split_data(df)

    X_train = train.drop(columns= target)
    y_train = train[target]

    X_validate = validate.drop(columns= target)
    y_validate = validate[target]

    X_test = test.drop(columns= target)
    y_test = test[target]
        
    # Have function print datasets shape
    print(f'''
    X_train -> {X_train.shape}
    X_validate -> {X_validate.shape}
    X_test -> {X_test.shape}''') 
    
    return X_train, y_train, X_validate, y_validate, X_test, y_test

1. Load the tips dataset.

In [4]:
df = data('tips')
df.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


a. Create a column named price_per_person. This should be the total bill divided by the party size.

In [5]:
df = df.rename(columns={'size':'number_of_people'})

In [6]:
df['price_per_person'] = df.total_bill / df.number_of_people

In [7]:
df.head()

,total_bill,tip,sex,smoker,day,time,number_of_people,price_per_person
1,16.99,1.01,Female,No,Sun,Dinner,2,8.495000
2,10.34,1.66,Male,No,Sun,Dinner,3,3.446667
3,21.01,3.50,Male,No,Sun,Dinner,3,7.003333
4,23.68,3.31,Male,No,Sun,Dinner,2,11.840000
5,24.59,3.61,Female,No,Sun,Dinner,4,6.147500


b. Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount? 

total_bill is a likely candidate

In [8]:
obj_cols = get_object_cols(df)

In [9]:
df_dummy=pd.get_dummies(columns=obj_cols, data=df)

In [10]:
df_dummy.head()

,total_bill,tip,number_of_people,price_per_person,sex_Female,sex_Male,smoker_No,smoker_Yes,day_Fri,day_Sat,day_Sun,day_Thur,time_Dinner,time_Lunch
1,16.99,1.01,2,8.495000,1,0,1,0,0,0,1,0,1,0
2,10.34,1.66,3,3.446667,0,1,1,0,0,0,1,0,1,0
3,21.01,3.50,3,7.003333,0,1,1,0,0,0,1,0,1,0
4,23.68,3.31,2,11.840000,0,1,1,0,0,0,1,0,1,0
5,24.59,3.61,4,6.147500,1,0,1,0,0,0,1,0,1,0


In [11]:
df_dummy.drop(columns = ['sex_Female', 'smoker_No', 'time_Dinner', 'day_Thur'], inplace=True)
df_dummy.head()

,total_bill,tip,number_of_people,price_per_person,sex_Male,smoker_Yes,day_Fri,day_Sat,day_Sun,time_Lunch
1,16.99,1.01,2,8.495000,0,0,0,0,1,0
2,10.34,1.66,3,3.446667,1,0,0,0,1,0
3,21.01,3.50,3,7.003333,1,0,0,0,1,0
4,23.68,3.31,2,11.840000,1,0,0,0,1,0
5,24.59,3.61,4,6.147500,0,0,0,0,1,0


In [12]:
train, validate, test = split_data(df_dummy)

In [13]:
X_train, y_train, X_validate, y_validate, X_test, y_test = X_y_split(df_dummy, 'tip')


    X_train -> (146, 9)
    X_validate -> (49, 9)
    X_test -> (49, 9)


In [14]:
X_train.columns.tolist()

['total_bill',
 'number_of_people',
 'price_per_person',
 'sex_Male',
 'smoker_Yes',
 'day_Fri',
 'day_Sat',
 'day_Sun',
 'time_Lunch']

In [18]:
from sklearn.preprocessing import MinMaxScaler

In [16]:
to_scale = X_train.columns.tolist()
to_scale

['total_bill',
 'number_of_people',
 'price_per_person',
 'sex_Male',
 'smoker_Yes',
 'day_Fri',
 'day_Sat',
 'day_Sun',
 'time_Lunch']

In [20]:
def scale_data(train, 
               validate, 
               test, 
               to_scale):

    train_scaled = train.copy()
    validate_scaled = validate.copy()
    test_scaled = test.copy()

    scaler = MinMaxScaler()

    scaler.fit(train[to_scale])

    train_scaled[to_scale] = scaler.transform(train[to_scale])
    validate_scaled[to_scale] = scaler.transform(validate[to_scale])
    test_scaled[to_scale] = scaler.transform(test[to_scale])
    
    return train_scaled, validate_scaled, test_scaled


In [21]:
X_train_scaled, X_validate_scaled, X_test_scaled = scale_data(X_train, X_validate, X_test, to_scale)

In [22]:
X_train_scaled.columns

Index(['total_bill', 'number_of_people', 'price_per_person', 'sex_Male',
       'smoker_Yes', 'day_Fri', 'day_Sat', 'day_Sun', 'time_Lunch'],
      dtype='object')

In [23]:
X_train_scaled.head()

,total_bill,number_of_people,price_per_person,sex_Male,smoker_Yes,day_Fri,day_Sat,day_Sun,time_Lunch
88,0.253214,0.2,0.422184,1.0,0.0,0.0,0.0,0.0,1.0
210,0.126492,0.2,0.217638,0.0,1.0,0.0,1.0,0.0,0.0
108,0.412305,0.2,0.678977,1.0,1.0,0.0,1.0,0.0,0.0
75,0.171717,0.2,0.290637,0.0,0.0,0.0,1.0,0.0,0.0
134,0.115014,0.2,0.199111,0.0,0.0,0.0,0.0,0.0,1.0


c. Use select k best to select the top 2 features for predicting tip amount. What are they?

In [24]:
kbest = SelectKBest(f_regression, k=2)

kbest.fit(X_train_scaled, y_train)

SelectKBest(k=2, score_func=<function f_regression at 0x1233e8430>)

In [25]:
kbest_results = pd.DataFrame(dict(p_value=kbest.pvalues_, f_score=kbest.scores_), 
                index=X_train_scaled.columns)

In [26]:
kbest_results.sort_values(by=['f_score'], ascending=False)

,p_value,f_score
total_bill,1.104393e-21,128.452877
number_of_people,2.231795e-10,46.622188
price_per_person,2.327229e-05,19.130794
sex_Male,2.644577e-02,5.029563
time_Lunch,4.388726e-02,4.133191
day_Sun,8.729862e-02,2.963733
day_Fri,8.049442e-01,0.061211
day_Sat,8.510392e-01,0.035393
smoker_Yes,9.005615e-01,0.015668


In [27]:
kbest_results.sort_values(by=['f_score'], ascending=False).head(2)

,p_value,f_score
total_bill,1.104393e-21,128.452877
number_of_people,2.231795e-10,46.622188


In [28]:
feature_mask = kbest.get_support()

X_train_scaled.columns[feature_mask]

Index(['total_bill', 'number_of_people'], dtype='object')

d. Use recursive feature elimination to select the top 2 features for tip amount. What are they?

In [29]:
model = LinearRegression()

rfe = RFE(model, n_features_to_select=2)

rfe.fit(X_train_scaled, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [30]:
rfe_ranking = pd.DataFrame({'rfe_ranking': rfe.ranking_},
                              index=X_train_scaled.columns)

In [31]:
rfe_ranking.sort_values(by=['rfe_ranking'], ascending=True)

,rfe_ranking
total_bill,1
price_per_person,1
number_of_people,2
day_Sun,3
sex_Male,4
day_Fri,5
smoker_Yes,6
time_Lunch,7
day_Sat,8


e. Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features you are selecting?

In [32]:
X_train_scaled.columns[SelectKBest(score_func=f_regression, k=2).fit(X_train_scaled, y_train).get_support()]

Index(['total_bill', 'number_of_people'], dtype='object')

In [33]:
X_train_scaled.columns[SelectKBest(score_func=f_regression, k=5).fit(X_train_scaled, y_train).get_support()]

Index(['total_bill', 'number_of_people', 'price_per_person', 'sex_Male',
       'time_Lunch'],
      dtype='object')

In [34]:
X_train_scaled.columns[RFE(LinearRegression(), n_features_to_select=2).fit(X_train_scaled, y_train).get_support()]

Index(['total_bill', 'price_per_person'], dtype='object')

In [35]:
X_train_scaled.columns[RFE(LinearRegression(), n_features_to_select=5).fit(X_train_scaled, y_train).get_support()]


Index(['total_bill', 'number_of_people', 'price_per_person', 'sex_Male',
       'day_Sun'],
      dtype='object')

2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [42]:

def select_kbest(X, y, k=2):
    '''
    will take in two pandas objects:
    X: a dataframe representing numerical independent features
    y: a pandas Series representing a target variable
    k: a keyword argument defaulted to 2 for the number of ideal features we elect to select
    
    return: a list of the selected features from the SelectKBest process
    '''
    kbest = SelectKBest(f_regression, k=k)
    kbest.fit(X, y)
    mask = kbest.get_support()
    return X.columns[mask]

3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [43]:
def show_features_rankings(X_train, rfe):
    """
    Takes in a dataframe and a fit RFE object in order to output the rank of all features
    """
    ranks = pd.DataFrame({'rfe_ranking': rfe.ranking_}
                        ,index = X_train.columns)
    
    ranks = ranks.sort_values(by="rfe_ranking", ascending=True)
    
    return ranks

In [44]:

def rfe(X, y, k=2):
    '''
    will take in two pandas objects:
    X: a dataframe representing numerical independent features
    y: a pandas Series representing a target variable
    k: a keyword argument defaulted to 2 for the number of ideal features we elect to select
    
    return: a list of the selected features from the recursive feature elimination process
        & a df of all rankings
    '''
   
    rfe = RFE(LinearRegression(), n_features_to_select=k)
 
    rfe.fit(X, y)
        
    features_to_use = X.columns[rfe.support_].tolist()
    
    all_rankings = show_features_rankings(X, rfe)
    
    return features_to_use, all_rankings

4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [36]:
#load swiss data
swiss= data('swiss')

In [37]:
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [38]:
target = "Fertility"

train, validate, test = split_data(swiss)

X_train = train.drop(columns=[target])
y_train = train[target]

X_validate = validate.drop(columns=[target])
y_validate = validate[target]

X_test = test.drop(columns=[target])
y_test = test[target]

In [39]:
to_scale = X_train.columns.tolist()

In [40]:
X_train_scaled, X_validate_scaled, X_test_scaled = scale_data(X_train, X_validate, X_test, to_scale)

In [45]:
select_kbest(X_train_scaled, y_train, 3).tolist()

['Education', 'Catholic', 'Infant.Mortality']

In [46]:
selected_features, all_rankings = rfe(X_train_scaled, y_train, 3)

print(selected_features)
all_rankings



['Agriculture', 'Education', 'Infant.Mortality']


,rfe_ranking
Agriculture,1
Education,1
Infant.Mortality,1
Catholic,2
Examination,3
